In [ ]:
import os
import json
import time
import numpy as np
from sklearn.metrics import pairwise


In [ ]:
def scale_transform_normalize(coords):
    """
    Parameters:
    coords (ndarray): array of (x,y,c) coordinates

    Returns:
    ndarray: coords scaled to 1x1 with center at (0,0)
    ndarray: confidence scores of each joint
    """
    coords, scores = coords[:,:,:-1], coords[:,:,-1]
    diff = coords.max(axis=1) - coords.min(axis=1)
    diff_max = np.max(diff, axis=0)
    mean = coords.mean(axis=1).reshape(coords.shape[0],1,coords.shape[-1])
    out = (coords - mean) / diff_max
    out/= np.linalg.norm(out, axis=2)[:, :, np.newaxis]
    out=out.reshape((out.shape[0], -1))
    
    return out, scores

In [ ]:
frame_folder_path='/content/drive/MyDrive/posedetection/json_video/'
files=[filename for filename in os.listdir(frame_folder_path)]
print(files)

['squat_000000000000_keypoints.json', 'squat_000000000001_keypoints.json', 'squat_000000000002_keypoints.json', 'squat_000000000003_keypoints.json', 'squat_000000000004_keypoints.json', 'squat_000000000005_keypoints.json', 'squat_000000000006_keypoints.json', 'squat_000000000007_keypoints.json', 'squat_000000000008_keypoints.json', 'squat_000000000009_keypoints.json', 'squat_000000000010_keypoints.json', 'squat_000000000011_keypoints.json', 'squat_000000000012_keypoints.json', 'squat_000000000013_keypoints.json', 'squat_000000000014_keypoints.json', 'squat_000000000015_keypoints.json', 'squat_000000000016_keypoints.json', 'squat_000000000017_keypoints.json', 'squat_000000000018_keypoints.json', 'squat_000000000019_keypoints.json', 'squat_000000000020_keypoints.json', 'squat_000000000021_keypoints.json', 'squat_000000000022_keypoints.json', 'squat_000000000023_keypoints.json', 'squat_000000000024_keypoints.json', 'squat_000000000025_keypoints.json', 'squat_000000000026_keypoints.json', 

In [ ]:
i=0
frame_range=24
rep_counter=0
while(i<=len(files)): #loop to cut video into frame set
  number_of_frames=len(files[i:i+frame_range])
  frames=files[i:i+frame_range]
  if(number_of_frames)!=frame_range:
    break
  # print(number_of_frames)
  keypoints=np.array([])
  # print(frames)
  match_counter = 0
  for frame in frames:  #loop to traverse through frame set

    f=open((frame_folder_path+frame),)
    raw_json=json.load(f)
    raw_keypoints=list(raw_json.items())[1][1][0]['pose_keypoints_2d']
    keypoints_dump=[]
    for keys in range(0,len(raw_keypoints),3):
      keypoints_dump.append(np.array([raw_keypoints[keys],raw_keypoints[keys+1],raw_keypoints[keys+2]]))
    keypoints=np.append(keypoints,keypoints_dump)

  # var keypoint contains keypoints from all frames in set
  # var checkpoint contains all keypoints from 0, 1/4, 1/2,3,4 and last frame from set of frames
  keypoints=keypoints.reshape(number_of_frames,18,3)
  checkpoints=np.array([keypoints[0],keypoints[number_of_frames//4],keypoints[number_of_frames//2],keypoints[3*number_of_frames//4],keypoints[number_of_frames-1]])
  #normalize checkpoints
  checkpoints,scores_cp=scale_transform_normalize(checkpoints)
  # normalize keypoints
  keypoints,scores_kp=scale_transform_normalize(keypoints)
  # print(keypoints.shape,checkpoints.shape)
  # print(keypoints[0].shape)
  similarity_list=pairwise.cosine_similarity(keypoints[0].reshape(1,-1),checkpoints)[0]
  matched_frame=(np.argmax(similarity_list),similarity_list[np.argmax(similarity_list)])
  if matched_frame[0]==0:
    match_counter+=1
  break
  



  
  
  




    


  
  i+=frame_range

In [ ]:
similarity_list

array([1.        , 0.99981209, 0.99957265, 0.99941444, 0.99949874])

0